# LSTM Sample 1 - Sin wave prediction

Inport the main libraries including the keras libraries we need. 
Note: keras has FutureWarning error message. That is OL


In [1]:
import numpy as np # linear algebra
import pandas # data processing, CSV file I/O (e.g. pandas.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda, Flatten, LSTM
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam, RMSprop
from keras.utils import np_utils

C:\Users\tgibbons\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Playing with arrays and data
Create a one dimensional array to represent the time series data. Then reformat it for LSTM.
See https://machinelearningmastery.com/reshape-input-data-long-short-term-memory-networks-keras/ for details.

In [3]:
from numpy import array
data = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
data2 = data.reshape((1, 10, 1))
print (data.shape)
print (data2.shape)

(10,)
(1, 10, 1)


In [4]:
print (data)
print (data2)

[0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
[[[0.1]
  [0.2]
  [0.3]
  [0.4]
  [0.5]
  [0.6]
  [0.7]
  [0.8]
  [0.9]
  [1. ]]]


In [5]:
data2D = array([
	[0.1, 1.0],
	[0.2, 0.9],
	[0.3, 0.8],
	[0.4, 0.7],
	[0.5, 0.6],
	[0.6, 0.5],
	[0.7, 0.4],
	[0.8, 0.3],
	[0.9, 0.2],
	[1.0, 0.1]])
	
data2Dshaped = data2D.reshape(1, 10, 2)

In [6]:
print (data2D)
print (data2Dshaped)

[[0.1 1. ]
 [0.2 0.9]
 [0.3 0.8]
 [0.4 0.7]
 [0.5 0.6]
 [0.6 0.5]
 [0.7 0.4]
 [0.8 0.3]
 [0.9 0.2]
 [1.  0.1]]
[[[0.1 1. ]
  [0.2 0.9]
  [0.3 0.8]
  [0.4 0.7]
  [0.5 0.6]
  [0.6 0.5]
  [0.7 0.4]
  [0.8 0.3]
  [0.9 0.2]
  [1.  0.1]]]


# Generate the samples from the sin() function
np.arrange generates numbers between 0 andt the number of samples
np.linspace(start value, end value, number of values

In [7]:
numberSamples = 1000
numericSamples = np.arange(numberSamples)
timeSamples = np.linspace(0,20,numberSamples)
print (numericSamples.shape)
print (numericSamples[:10])
print (timeSamples.shape)
print (timeSamples[:10])
sinSamples = np.sin(timeSamples)
print (sinSamples[:10])

(1000,)
[0 1 2 3 4 5 6 7 8 9]
(1000,)
[0.         0.02002002 0.04004004 0.06006006 0.08008008 0.1001001
 0.12012012 0.14014014 0.16016016 0.18018018]
[0.         0.02001868 0.04002934 0.06002396 0.07999452 0.09993302
 0.11983146 0.13968188 0.15947632 0.17920684]


In [8]:
from matplotlib import pyplot as plt
plt.plot(sinSamples)
#plt.ylabel('some numbers')
plt.xlabel('Sine wave')
plt.show()

# Setup Training and Testing data

Split the data into a training sequence and a testing sequence

In [19]:
# split into train and test sets
train_size = int(len(sinSamples) * 0.70)
test_size = len(sinSamples) - train_size
sin_train = sinSamples[0:train_size]
sin_test  = sinSamples[train_size:len(sinSamples)]
print ("train shape : " , sin_train.shape)
print ("test shape : " , sin_test.shape)




train shape :  (700,)
test shape :  (300,)


The above sets up the 1-d sin data for input into a LSTM system 


In [20]:
def to_sequences(seq_size, obs):
    x = []
    y = []

    for i in range(len(obs)-SEQUENCE_SIZE-1):
        #print(i)
        window = obs[i:(i+SEQUENCE_SIZE)]
        after_window = obs[i+SEQUENCE_SIZE]
        window = [[x] for x in window]
        #print("{} - {}".format(window,after_window))
        x.append(window)
        y.append(after_window)
        
    return np.array(x),np.array(y)
    

In [21]:
SEQUENCE_SIZE = 10
x_train,y_train = to_sequences(SEQUENCE_SIZE,sin_train)
x_test,y_test = to_sequences(SEQUENCE_SIZE,sin_test)

print("Shape of training set: {}".format(x_train.shape))
print("Shape of test set: {}".format(x_test.shape))

Shape of training set: (689, 10, 1)
Shape of test set: (289, 10, 1)


# Build the Neural Network model in Keras

Build model and then train on training data and test on testin data
<br>
Model breaks long sequence up into many small 10 value sequences. Input is single input at time t and predicts single output at time t+1. There are 689 sequences for training, each 10 values long.



In [22]:
print('Build model...')
model = Sequential()
model.add(LSTM(64, input_shape=(None, 1)))
model.add(Dense(32))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

#monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
print('Train...')



Build model...
Train...


In [23]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),verbose=2,epochs=100)

Train on 689 samples, validate on 289 samples
Epoch 1/100
 - 1s - loss: 0.1927 - val_loss: 0.0356
Epoch 2/100
 - 0s - loss: 0.0126 - val_loss: 0.0065
Epoch 3/100
 - 0s - loss: 0.0046 - val_loss: 0.0041
Epoch 4/100
 - 0s - loss: 0.0038 - val_loss: 0.0042
Epoch 5/100
 - 0s - loss: 0.0036 - val_loss: 0.0040
Epoch 6/100
 - 0s - loss: 0.0035 - val_loss: 0.0040
Epoch 7/100
 - 0s - loss: 0.0034 - val_loss: 0.0037
Epoch 8/100
 - 0s - loss: 0.0032 - val_loss: 0.0033
Epoch 9/100
 - 0s - loss: 0.0031 - val_loss: 0.0031
Epoch 10/100
 - 0s - loss: 0.0030 - val_loss: 0.0029
Epoch 11/100
 - 0s - loss: 0.0028 - val_loss: 0.0030
Epoch 12/100
 - 0s - loss: 0.0025 - val_loss: 0.0026
Epoch 13/100
 - 0s - loss: 0.0023 - val_loss: 0.0025
Epoch 14/100
 - 0s - loss: 0.0021 - val_loss: 0.0021
Epoch 15/100
 - 0s - loss: 0.0019 - val_loss: 0.0018
Epoch 16/100
 - 0s - loss: 0.0016 - val_loss: 0.0017
Epoch 17/100
 - 0s - loss: 0.0014 - val_loss: 0.0014
Epoch 18/100
 - 0s - loss: 0.0011 - val_loss: 0.0010
Epoch 19/

In [24]:
trainPredict = model.predict(x_train)
testPredict = model.predict(x_test)


# Plot original values and predictions

Original (truth) values are shown in dotted line. Results of prediction of training data shown in orange while testing data is shown in green.

In [25]:
from matplotlib import pyplot as plt
# plot baseline and predictions
plt.plot(sinSamples, linestyle=':')

plt.plot(trainPredict)
list = np.arange(len(trainPredict)+10,len(trainPredict)+len(testPredict)+10)
plt.plot(list,testPredict)